<a href="https://colab.research.google.com/github/LunaticRuri/yonsei-research-assistant/blob/main/Lora_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 텍스트이해와 인공지능 - LoRA 파인튜닝 노트북
참고하면 좋을 자료들:
- 수업 시간에 다룬 LoRA 파인튜닝 https://colab.research.google.com/drive/184eSkGiDYlVlRBZjh4kdoigTnqkb5-Jx?usp=sharing

- Introduction to LoRA Tuning using PEFT from Hugging Face.
Fine-tune a Foundational Model effortless
by Pere Martra https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/5-Fine%20Tuning/LoRA_Tuning_PEFT.ipynb


In [4]:
#@title 1. Install dependencies
#@markdown 혹시 몰라 깃헙 폴더 또한 클론하고 있습니다. 데이터 전처리 및 모델 훈련 영역에서 python 스크립트를 실행해야 하는 일이 생기면, 해당 레포의 utils/폴더 내에 py 파일들을 넣어 주시고, 이 코랩 내에서 !python을 통해 실행하시면 될 것 같습니다.
from IPython.display import clear_output
!git clone https://github.com/LunaticRuri/yonsei-research-assistant.git #추후 데이터 전처리를 위해 레포지토리 다운로드

!pip install huggingface
#@markdown (깔끔함을 위해... 추후 학습에 필요한 라이브러리들은 이 셀에 추가해 주시면 됩니다. <br><br> * 최종 출력은 스크롤압박 방지를 위해 최하단에서 clear_output()을 통해 지우고 있으나, 이 부분 마음에 안 드시면 주석 처리하시면 됩니다.)
clear_output() #지저분하니 지웁니다

In [2]:
#@title 1-2. Google Drive 마운트
#@markdown 추후 체크포인트 및 모델 파일들은 구글 드라이브에 저장하면 좋습니다
#@markdown <br> "/content/drive/MyDrive" 로 구글 드라이브에 접근 가능합니다.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#@title 2. KRAG 데이터셋을 data 폴더에 unzip
import gdown
import os
from shutil import rmtree
# 혼선방지를 위해 data폴더 삭제
rmtree("data", ignore_errors=True)
os.makedirs("data", exist_ok=True)

file_id = "1_HG7P_JJ3LoOuaPEA3aNJZnAOH-hywPO"
output = "data/KRAG.zip" # 저장 위치 및 저장할 파일 이름
gdown.download(id=file_id, output=output, quiet=False)

#KRAG 데이터셋을 data 폴더에 다운로드받습니다
print("📂 데이터셋 압축 해제중...")
!unzip -qq '/content/data/KRAG.zip' -d '/content/data/'
print("✔️ 데이터셋 압축 해제 완료")


Downloading...
From (original): https://drive.google.com/uc?id=1_HG7P_JJ3LoOuaPEA3aNJZnAOH-hywPO
From (redirected): https://drive.google.com/uc?id=1_HG7P_JJ3LoOuaPEA3aNJZnAOH-hywPO&confirm=t&uuid=786ed2f2-eea4-4407-a64d-d82b670bf927
To: /content/data/KRAG.zip
100%|██████████| 2.04G/2.04G [00:29<00:00, 68.9MB/s]


📂 데이터셋 압축 해제중...
✔️ 데이터셋 압축 해제 완료


In [7]:
from datasets import load_dataset

dataset = load_dataset("json",
                       data_files=
                        {"train": "/content/data/K-RAG/train_dataset/train.json",
                         "test": "/content/data/K-RAG/train_dataset/test.json",
                        "valid": "/content/data/K-RAG/train_dataset/valid.json"} )

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

In [8]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'keyphrases', 'key_sentences', 'gold_passage_id'],
        num_rows: 54224
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'keyphrases', 'key_sentences', 'gold_passage_id'],
        num_rows: 6779
    })
    valid: Dataset({
        features: ['id', 'question', 'answer', 'keyphrases', 'key_sentences', 'gold_passage_id'],
        num_rows: 6778
    })
})
